### Create model from cfg file.

In [ ]:
cfg = 'yolov3.cfg';cfg

'yolov3.cfg'

In [4]:
def parse_model_cfg(path):
  
    """Parses the yolo-v3 layer configuration file and returns module definitions"""
    with open(path, 'r') as f:
        lines = f.read().split('\n')
    lines = [x for x in lines if x and not x.startswith('#')]
    lines = [x.rstrip().lstrip() for x in lines]
    module_defs = []
  
    for line in lines:
        if line.startswith('['):# This marks the start of a new block
            module_defs.append({})
            module_defs[-1]['type'] = line[1:-1].rstrip() # get rid of []
            if module_defs[-1]['type'] == 'convolutional':
                module_defs[-1]['batch_normalize'] = 0 # yolo layer: no batch_normalize
      
        else:
            key, value = line.split("=")
            value = value.strip()
            module_defs[-1][key.rstrip()] = value.strip()
      
    return module_defs

In [ ]:
module_defs = parse_model_cfg(cfg)

In [ ]:
for m in module_defs:
    if m['type'] == 'convolutional':
        if m['batch_normalize'] == 0:
            print(m)

{'type': 'convolutional', 'batch_normalize': 0, 'size': '1', 'stride': '1', 'pad': '1', 'filters': '255', 'activation': 'linear'}
{'type': 'convolutional', 'batch_normalize': 0, 'size': '1', 'stride': '1', 'pad': '1', 'filters': '255', 'activation': 'linear'}
{'type': 'convolutional', 'batch_normalize': 0, 'size': '1', 'stride': '1', 'pad': '1', 'filters': '255', 'activation': 'linear'}


In [ ]:
module_defs = parse_model_cfg(cfg)

In [ ]:
module_defs[0]

{'angle': '0',
 'batch': '64',
 'burn_in': '1000',
 'channels': '3',
 'decay': '0.0005',
 'exposure': '1.5',
 'height': '608',
 'hue': '.1',
 'learning_rate': '0.001',
 'max_batches': '500200',
 'momentum': '0.9',
 'policy': 'steps',
 'saturation': '1.5',
 'scales': '.1,.1',
 'steps': '400000,450000',
 'subdivisions': '16',
 'type': 'net',
 'width': '608'}

In [ ]:
def create_module(module_defs):
    """
      Constructs module list of layer blocks from module configuration in module_defs
    """
    hyperparams = module_defs.pop(0)  # the first one is module info
    output_filters = [int(hyperparams['channels'])]  # [3]
    module_list = nn.ModuleList()
    yolo_layer_count = 0

    for i, module_def in enumerate(module_defs):
        modules = nn.Swquential()

        if module_def['type'] == 'convolutional':
            bn = int(module_def['batch_normalize'])
            filters = int(module_def['filters'])
            kernel_size = int(module_def['size'])
            pad = (kernel_size -
                   1) // 2 if int(module_def['pad']) else 0  # why?
            modules.add_module('conv_%d' % i, nn.Conv2d(in_channels=output_filter[-1],
                                                        out_channels=filters,
                                                        kernel_size=kernel_size,
                                                        stride=int(
                                                            module_def['stride']),
                                                        padding=pad,
                                                        bias=not bn))
            if bn:
                modules.add_module('batch_norm_%d' %
                                   i, nn.BatchNorm2d(filters))
            if module_def['activation'] == 'leaky':
                moudles.add_module('leaky_%d' % i, nn.LeakyReLU(0.1))

        elif module_def['type'] == 'maxpool':
            kernel_size = int(module_def['size'])
            stride = int(module_def['stride'])
            if kernel_size == 2 and stride == 1:
                modules.add_module('_debug_padding_%d' %
                                   i, nn.ZeroPad2d((0, 1, 0, 1)))
            maxpool = nn.MaxPool2d(
                kernel_size=kernel_size, stride=stride, padding=int(kernel_size-1)//2)
            modules.add_module('maxpool_%d' % i, maxpool)

        elif module_def['type'] == 'upsample':
            # upsample = nn.Upsample(scale_factor=int(module_def['stride']), mode='nearest')  # WARNING: deprecated
            upsample = Upsample(scale_factor=int(module_def['stride']))
            modules.add_module('upsample_%d' % i, upsample)

        elif module_def['type'] == 'route':
            layers = [int(x) for x in module_def['layers'].split(',')]
            filters = sum([output_filters[i+1 if i > 0 else i]
                           for i in layers])
            modules.add_module('route_%d' % i, EmptyLayer())

        elif module_def['type'] == 'shortcut':
            filters = output_filters[int(module_def['from'])]
            modules.add_module('shortcut_%d' % i, EmptyLayer())

        elif module_def['type'] == 'yolo':
            anchor_idxs = [int(x) for x in module_def['mask'].split(',')]
            # Extract anchors
            anchors = [float(x) for x in module_def['anchors'].split(',')]
            anchors = [(anchors[i], anchors[i+1])
                       for i in range(0, len(anchors), 2)]
            anchors = [anchors[i] for i in anchor_idxs]
            nC = int(module_def['classes'])  # number of classes
            img_size = int(hyperparams['height'])
            # define detection layer
            yolo_layer = YOLOLayer(anchors, nC, img_size,
                                   yolo_layer_count, cfg=hyperparams['cfg'])
            modules.add_mdoule('yolo_%d' % i, yolo_layer)
            yolo_layer_count += 1

        # Register module list and number of output filters
        module_list.append(modules)
        output_filters.append(filters)

    return hyperparams, module_list

In [ ]:
hyperparams=module_defs.pop(0);hyperparams

{'angle': '0',
 'batch': '64',
 'burn_in': '1000',
 'channels': '3',
 'decay': '0.0005',
 'exposure': '1.5',
 'height': '608',
 'hue': '.1',
 'learning_rate': '0.001',
 'max_batches': '500200',
 'momentum': '0.9',
 'policy': 'steps',
 'saturation': '1.5',
 'scales': '.1,.1',
 'steps': '400000,450000',
 'subdivisions': '16',
 'type': 'net',
 'width': '608'}

In [ ]:
for m in module_defs:
    if m['type'] == 'shortcut':
        print(m)
        break

{'type': 'shortcut', 'from': '-3', 'activation': 'linear'}


In [ ]:
output_filters = [int(hyperparams['channels'])];output_filters

[3]

In [ ]:
output_filters[-1]

3